In [1]:
import sys,os

import requests
import datetime,pandas,dateutil
from bs4 import BeautifulSoup
import mrigstatics,mrigutilities
import pandas as pd
import time
import interface.mrigxl as mxl
import QuantLib as ql

C:\anaconda\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [52]:
today = ql.Date(29, ql.September, 2023)
future = ql.Date(22, ql.October, 2023)
reference_date = today
ql.Settings.instance().evaluationDate = today

sql = "select (curvedate + (tenor::TEXT)::INTERVAL) as date, yield from yieldcurve where curve = 'INR' and curvedate='2023-9-29'"
engine = mrigutilities.sql_engine()

rates_df = pd.read_sql(sql,engine)
rates_df['date'] = pd.to_datetime(rates_df['date'],format='%Y-%m-%d').apply(lambda x: x.date())
rates_df = rates_df.sort_values(by='date')
#        print(rates_df)
spot_yields = list(rates_df['yield']/100)
spot_dates = list(rates_df['date'])
spot_dates = [ql.Date(dt.day,dt.month,dt.year) for dt in spot_dates]
#        spot_dates = mrigutilities.get_date_vector([list(rates_df['curvedate']),list(rates_df['tenor'])])
#        print(spot_dates)
spot_dates.insert(0,today)
spot_yields.insert(0,0.0)
spot_rates = [spot_dates,spot_yields]       

zc =  ql.ZeroCurve(spot_rates[0],
                                 spot_rates[1],
                                 ql.Thirty360(),
                                 ql.India(),
                                 ql.Linear(),
                                 ql.Compounded,
                                 ql.Annual)
fc =  ql.ForwardCurve(spot_rates[0],
                                 spot_rates[1],
                                 ql.Thirty360(),
                                 ql.India())

izc = ql.ImpliedTermStructure(ql.YieldTermStructureHandle(zc),future)


In [55]:

option = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
                                   ql.EuropeanExercise(ql.Date(26, ql.October, 2023)))

u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)
        
riskFreeCurve = ql.FlatForward(0, ql.TARGET(),ql.QuoteHandle(r), ql.Thirty360())
irfc = ql.ImpliedTermStructure(ql.YieldTermStructureHandle(riskFreeCurve),future)

volatility = ql.BlackConstantVol(0, ql.TARGET(),ql.QuoteHandle(sigma), ql.Actual360())


process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                         ql.YieldTermStructureHandle(zc),
                                         ql.BlackVolTermStructureHandle(volatility))
engine = ql.AnalyticEuropeanEngine(process)

option.setPricingEngine(engine)

print(today,option.NPV())


ql.Settings.instance().evaluationDate = future

print(future,option.NPV())



September 29th, 2023 2.2585921877279027
October 22nd, 2023 0.8048312261608088


In [43]:
ql.Settings.instance().evaluationDate = today
riskFreeCurve.referenceDate()

curve_today = []
for day in range(0,180,5):
    zr = riskFreeCurve.zeroRate(future+day,ql.Thirty360(),ql.Compounded)
    fr = irfc.zeroRate(future+day,ql.Thirty360(),ql.Compounded)
    curve_today.append((today,today+day,zr.rate(),fr.rate()))
curve_today = pd.DataFrame(curve_today,columns=['today','future','rate','frate'])
print(curve_today)
# curve_today.plot()

# ql.Settings.instance().evaluationDate = future
print('------------')
curve_future = []
for day in range(0,180,5):
    zr = izc.zeroRate(future + day,ql.Thirty360(),ql.Compounded)
    fr = zc.zeroRate(future + day,ql.Thirty360(),ql.Compounded)
    curve_future.append((future,future+day,zr.rate(),fr.rate()))
curve_future = pd.DataFrame(curve_future,columns=['today','future','rate','frate'])
print(curve_future)
# curve_future.plot()

                   today                future     rate    frate
0   September 29th, 2023  September 29th, 2023  0.01005  0.01005
1   September 29th, 2023     October 4th, 2023  0.01005  0.01005
2   September 29th, 2023     October 9th, 2023  0.01005  0.01005
3   September 29th, 2023    October 14th, 2023  0.01005  0.01005
4   September 29th, 2023    October 19th, 2023  0.01005  0.01005
5   September 29th, 2023    October 24th, 2023  0.01005  0.01005
6   September 29th, 2023    October 29th, 2023  0.01005  0.01005
7   September 29th, 2023    November 3rd, 2023  0.01005  0.01005
8   September 29th, 2023    November 8th, 2023  0.01005  0.01005
9   September 29th, 2023   November 13th, 2023  0.01005  0.01005
10  September 29th, 2023   November 18th, 2023  0.01005  0.01005
11  September 29th, 2023   November 23rd, 2023  0.01005  0.01005
12  September 29th, 2023   November 28th, 2023  0.01005  0.01005
13  September 29th, 2023    December 3rd, 2023  0.01005  0.01005
14  September 29th, 2023 